# importing claude

## low level api

In [1]:
import boto3
import json

# Initialize the Bedrock Runtime client
bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1' 
)

#low level api, calling to claude directly
def invoke_claude(prompt: str) -> str:
    # Model ID for Claude 3.5 Sonnet
    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
    
    # Request body
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "messages": [
            {
                "role": "user",
                "content": prompt}] 
    })
    
    try:
        response = bedrock.invoke_model(
            modelId=model_id,
            body=body
        )
        
        # Parse response
        response_body = json.loads(response['body'].read())
        return response_body['content'][0]['text']
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

In [2]:
prompt = "Hello, how are you?"
response = invoke_claude(prompt)
print(response)

Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you. How can I help you today?


## using langchain to call claude

### importing libraries from langchain

In [10]:
#use langchain to use claude
!pip install boto3
!pip install langchain-aws
!pip install langchain-community boto3
!pip install boto3 requests requests-aws4auth
!pip install pymongo
!pip install google-search-results sagemaker
!pip install boto3 nltk

#chat models
from langchain_aws import ChatBedrock
from langchain_aws import ChatBedrockConverse

#LLMs
from langchain_aws import BedrockLLM
from langchain_community.llms import AmazonAPIGateway
from langchain_aws import SagemakerEndpoint

#embedding models
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import ContentHandlerBase

#document loaders
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
from langchain_community.document_loaders import AmazonTextractPDFLoader

#vector stores
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_community.vectorstores import DocumentDBVectorSearch

from langchain_aws.vectorstores.inmemorydb import InMemoryVectorStore

#retrievers
from langchain_aws import AmazonKendraRetriever
from langchain_aws import AmazonKnowledgeBasesRetriever

#memory
from langchain_community.chat_message_histories import DynamoDBChatMessageHistory

#graphs
from langchain_community.graphs import NeptuneGraph
from langchain_community.graphs import NeptuneAnalyticsGraph
from langchain_community.chains.graph_qa.neptune_cypher import NeptuneOpenCypherQAChain

from langchain_community.graphs import NeptuneRdfGraph
from langchain_community.chains.graph_qa.neptune_sparql import NeptuneSparqlQAChain

#callbacks
from langchain_community.callbacks.bedrock_anthropic_callback import BedrockAnthropicTokenUsageCallbackHandler
from langchain_community.callbacks import SageMakerCallbackHandler

#chains
# from langchain_experimental.comprehend_moderation import AmazonComprehendModerationChain

In [14]:
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 200}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs=model_parameter, 
    beta_use_converse_api=True
)

bedrock_llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'c16e6162-4429-4acc-8425-23ec919d9a60', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 10 Dec 2024 13:36:52 GMT', 'content-type': 'application/json', 'content-length': '214', 'connection': 'keep-alive', 'x-amzn-requestid': 'c16e6162-4429-4acc-8425-23ec919d9a60'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [405]}}, id='run-1069943c-b85a-47e8-88a1-b3acb57c7ad0-0', usage_metadata={'input_tokens': 8, 'output_tokens': 12, 'total_tokens': 20})

### memory

In [ ]:
#memory
vds = InMemoryVectorStore.from_documents(
            chunks,
            embeddings,
            redis_url="rediss://cluster_endpoint:6379/ssl=True ssl_cert_reqs=none",
            vector_schema=vector_schema,
            index_name=INDEX_NAME,
        )

# langchain course notes

## langchain: models, prompts and parsers

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # Read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

# Chat API : OpenAI
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

get_completion("What is 1+1?")

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
style = """American English \
in a calm and respectful tone
"""
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```"""
response = get_completion(prompt)
response

# Chat API : LangChain
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0.0, model=llm_model)

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```"""
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

customer_style = """American English \
in a calm and respectful tone
"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)
customer_response = chat(customer_messages)
print(customer_response.content)

service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)
service_response = chat(service_messages)
print(service_response.content)

# Output Parsers
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""
review_template = """\
For the following text, extract the following information:
gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.
delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.
price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.
Format the output as JSON with the following keys:
gift
delivery_days
price_value
text: {text}
"""
prompt_template = ChatPromptTemplate.from_template(review_template)
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

# Parse the LLM output string into a Python dictionary
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

review_template_2 = """\
For the following text, extract the following information:
gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.
delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.
price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.
text: {text}
{format_instructions}
"""
prompt = ChatPromptTemplate.from_template(template=review_template_2)
messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)
response = chat(messages)
print(response.content)
output_dict = output_parser.parse(response.content)
output_dict


## Langchain: memory

In [11]:
#NOTE - work out how to import claude to be functioning
#llm = claude () smh

from langchain.chains import ConversationChain

#remembers everything
from langchain.memory import ConversationBufferMemory

#only keeps window of memory
from langchain.memory import ConversationBufferWindowMemory
memory=ConversationBufferWindowMemory(k=1) #specifies want to remember one conversational exchange (one input one output)
#only remembers most recent conversational items

#memory limits number of token saved
#if decrease max token limit - chops off earlier parts of conversation
from langchain.memory import ConversationTokenBufferMemory
memory=ConversationTokenBufferMemory(llm=llm, max_token_limit = 50) #change max token number
#need to specify llm as different llms have different ways of counting tokens

#let LLM write summary of conversation history and set that to be memory
from langchain.memory import ConversationSummaryBufferMemory
memory=ConversationTokenBufferMemory(llm=llm, max_token_limit = 100) #where max token limit is less than total number of tokens in history -> summarises, otherwise outputs full history

conversastion = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True #or False - play around once works
)

#outputs memory
memory.load_memory_variables({})

#from course exact code:

# import os
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())  # Read local .env file

# import warnings
# warnings.filterwarnings('ignore')

# # Account for deprecation of LLM model
# import datetime
# # Get the current date
# current_date = datetime.datetime.now().date()

# # Define the date after which the model should be set to "gpt-3.5-turbo"
# target_date = datetime.date(2024, 6, 12)

# # Set the model variable based on the current date
# if current_date > target_date:
#     llm_model = "gpt-3.5-turbo"
# else:
#     llm_model = "gpt-3.5-turbo-0301"

# from langchain.chat_models import ChatOpenAI
# from langchain.chains import ConversationChain
# from langchain.memory import ConversationBufferMemory

# llm = ChatOpenAI(temperature=0.0, model=llm_model)
# memory = ConversationBufferMemory()
# conversation = ConversationChain(
#     llm=llm, 
#     memory=memory,
#     verbose=True
# )
# conversation.predict(input="Hi, my name is Andrew")
# conversation.predict(input="What is 1+1?")
# conversation.predict(input="What is my name?")
# print(memory.buffer)
# memory.load_memory_variables({})

# memory = ConversationBufferMemory()
# memory.save_context({"input": "Hi"}, {"output": "What's up"})
# print(memory.buffer)
# memory.load_memory_variables({})
# memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
# memory.load_memory_variables({})

# from langchain.memory import ConversationBufferWindowMemory
# memory = ConversationBufferWindowMemory(k=1)
# memory.save_context({"input": "Hi"}, {"output": "What's up"})
# memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
# memory.load_memory_variables({})

# llm = ChatOpenAI(temperature=0.0, model=llm_model)
# memory = ConversationBufferWindowMemory(k=1)
# conversation = ConversationChain(
#     llm=llm, 
#     memory=memory,
#     verbose=False
# )
# conversation.predict(input="Hi, my name is Andrew")
# conversation.predict(input="What is 1+1?")
# conversation.predict(input="What is my name?")

# from langchain.memory import ConversationTokenBufferMemory
# from langchain.llms import OpenAI
# llm = ChatOpenAI(temperature=0.0, model=llm_model)
# memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
# memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
# memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
# memory.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})
# memory.load_memory_variables({})

# from langchain.memory import ConversationSummaryBufferMemory
# schedule = "There is a meeting at 8am with your product team. \
# You will need your powerpoint presentation prepared. \
# 9am-12pm have time to work on your LangChain \
# project which will go quickly because Langchain is such a powerful tool. \
# At Noon, lunch at the italian resturant with a customer who is driving \
# from over an hour away to meet you to understand the latest in AI. \
# Be sure to bring your laptop to show the latest LLM demo."

# memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
# memory.save_context({"input": "Hello"}, {"output": "What's up"})
# memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
# memory.save_context({"input": "What is on the schedule today?"}, 
#                     {"output": f"{schedule}"})
# memory.load_memory_variables({})
# conversation = ConversationChain(
#     llm=llm, 
#     memory=memory,
#     verbose=True
# )
# conversation.predict(input="What would be a good demo to show?")
# memory.load_memory_variables({})


/tmp/ipykernel_24783/3104868410.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferWindowMemory(k=1) #specifies want to remember one conversational exchange (one input one output)


NameError: name 'llm' is not defined

## Langchain: chain

### single chain

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

#import some pandas dataframe
#df = pd.read_csv('data.csv')

#some prompt
#prompt=ChatPromptTemplate.from_template(' ') eg:

prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain = LLMChain(llm=llm,prompt=prompt)
product = "Queen Size Sheet Set"
chain.run(product)


### multiple chains - single input output

In [ ]:
#for multiple chains, where each is a single input and single output
from langchain.chains import SimpleSequentialChain

first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain_one = LLMChain(llm=llm,prompt=first_prompt)

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
overall_simple_chain.run(product)

### multiple chains - multiple input outputs

In [ ]:
#if multiple inputs or outputs
from langchain.chains import SequentialChain
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)

# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# chain 2: input= English_Review (output from chain 1) and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

review = df.Review[5]
overall_chain(review)

### Router Chain - multiple subchains specialised for specific input

In [5]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""


prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

from langchain.chains.router import MultiPromptChain # used when routing between different types of route templates
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser 
#LLMRouterChain uses LLM itself to route between different sub chains - where description and name set above will be used
#RouterOutputParser parses LLM output into dictionary
from langchain.prompts import PromptTemplate


destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

#when model cant decide which prompt/chain to use
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
) #can add different destinations not mentioned in prompt template above

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

#can ask any question eg physics
chain.run("What is black body radiation?")


## Langchain: Question and Answer from documents

combines LLMs with data they werent initially trained on - more flexible and adaptable to use case

In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI

file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

from langchain.indexes import VectorstoreIndexCreator

#pip install docarray

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch #vector store class
).from_loaders([loader])

query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

llm_replacement_model = OpenAI(temperature=0, 
                               model='gpt-3.5-turbo-instruct')

response = index.query(query, 
                       llm = llm_replacement_model)

display(Markdown(response))

### embeddings

LLMs can only inspect a few thousand words at a time.if have large documents - use embeddings and vector stores.Embeddings - create numerical representations for pieces of text, pieces of text w similar content has similar vector. Vector database - store vector representations - populate w chunks of text from incoming document - split up document into smaller chunks - only pass most relevant ones to LLM, embedding for each chunk created and stored in vector data base - what happens when create index - can use index in runtime to find piece of text most relevant to incoming query - when query comes in - creat emebedding for query, compare w all vectors in database, pick the n most similar

In [ ]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)
docs = loader.load()
from langchain.embeddings import OpenAIEmbeddings
docs[0]

#because document not that large can create embedding directly

embeddings = OpenAIEmbeddings() #need to use relevant embedding class not from open ai


embed = embeddings.embed_query("Hi my name is Harrison")

#creates overall vector store
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)


query = "Please suggest a shirt with sunblocking"
#returns all documents that are similar to query
docs = db.similarity_search(query)

### Question and Answering

In [ ]:
retriever = db.as_retriever() #takes in query and returns documents

#if doing by hand - combines documents into single piece of text, join all page content into variable
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 
display(Markdown(response))

#all steps can be shortened into one line using langchain
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", #use stuff method - put all into one prompt and get one response
    retriever=retriever, 
    verbose=True
)

query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

response = qa_stuff.run(query)
display(Markdown(response))

#or can do it more simpler too:
response = index.query(query, llm=llm)

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

## Langchain: evaluation

In [7]:
# sae question answering code as before
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

llm = ChatOpenAI(temperature=0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs={
        "document_separator": "<<<<>>>>>"
    }
)

# Coming up with test datapoints that you want to evaluate
data[10]
data[11]

# Hard-coded examples
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# LLM-Generated examples
from langchain.evaluation.qa import QAGenerateChain

example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model)) #create chain using LLM

# The warning below can be safely ignored
new_examples = example_gen_chain.apply_and_parse( #apply output parser to result, want back dictionary with input output pair - not just string
    [{"doc": t} for t in data[:5]]
)

new_examples[0]
data[0]

# Combine examples
examples += new_examples

qa.run(examples[0]["query"])

# Manual Evaluation
import langchain
langchain.debug = True

qa.run(examples[0]["query"])  #outputs step by step LLM analysis

# Turn off the debug mode
langchain.debug = False

# LLM-assisted evaluation
predictions = qa.apply(examples)

from langchain.evaluation.qa import QAEvalChain

llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

graded_outputs = eval_chain.evaluate(examples, predictions)

for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

graded_outputs[0]

ModuleNotFoundError: No module named 'langchain_community'

## Langchain: agents

In [18]:
# Built-in LangChain tools
!pip install -U wikipedia
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

#need temp=0 as llm is reasoning engine - need as precise as possible, no randomness
llm = ChatOpenAI(temperature=0, model=llm_model)

#llm-math = chain, uses LLM w calculator to do math problems
#allows you to use api calls to wikipedia
tools = load_tools(["llm-math", "wikipedia"], llm=llm)


agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, #chat = agent that is optimised to work w chat models
    #react = prompting tehcnique designe dto get best reasoning performance out of language models
    handle_parsing_errors=True, 
    verbose=True
)

#llm math question
agent("What is the 25% of 300?")

# Wikipedia example
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question)

# Python Agent
# using llm to write code, then executing that code
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(), #repl = way to interact with code, agent can execute code with repl, passed back to agent so it can decide what to do next
    verbose=True
)

customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff", "Fusion"], 
                 ["Trance", "Former"],
                 ["Jen", "Ayai"]]

agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")

# View detailed outputs of the chains
langchain.debug = True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")
langchain.debug = False

# Define your own tool
from langchain.agents import tool #turns any function into tool agent can use
from datetime import date

@tool
def time(text: str) -> str:
    """Returns today's date. Use this for any \
    questions related to knowing today's date. \
    The input should always be an empty string, \
    and this function will always return today's \
    date - any date mathematics should occur \
    outside this function."""
    return str(date.today())

agent = initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=c91c99b07e9a1a6886a6f8e92d75e05687816177f03ba002be46c658bc86e02f
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


ValueError: '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain/agents/agent_toolkits' is not in the subpath of '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core' OR one path is relative and the other is absolute.

# AWS course notes

## first agent

### creating agent

In [12]:

# sdk for aws running in python
import boto3

# Create a Bedrock agent



model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 200}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs=model_parameter, 
    beta_use_converse_api=True
)

bedrock_llm.invoke("hello")
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-east-1') #create client contact

create_agent_response = bedrock_agent.create_agent(
    agentName='automated-retail-replenishment-analysis-agent', #NOTE - TRY CHANGING NAME FOR AGENT
    foundationModel='anthropic.claude-3-5-sonnet-20240620-v1:0', 
    instruction="""You are an advanced AI agent acting as a front line customer support agent.""", #CHANGE THIS TO BE MORE DESCRIPTIVE
    agentResourceRoleArn=roleArn #defines for agent what services it can get access to - least privledge - only access to foundation model
)
create_agent_response

#lifecycle = creating -> not prepared -> prepared
#then can create alias for it - production end point we can use - stability

# Retrieve the agent ID
agentId = create_agent_response['agent']['agentId']


sh: ./ro_shared_data/reset.sh: No such file or directory


KeyError: 'BEDROCKAGENTROLE'

In [13]:
from langchain_aws import ChatBedrock
from langchain.agents import initialize_agent, Tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMMathChain
import boto3

# Step 1: Define the LLM
model_parameter = {"temperature": 0.0, "top_p": 0.5, "max_tokens_to_sample": 200}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

# Initialize ChatBedrock LLM
boto3_bedrock = boto3.client("bedrock")  # Ensure your AWS credentials are configured
bedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs=model_parameter,
    beta_use_converse_api=True,
)

# Step 2: Define a Tool (Optional Example Tool: Calculator)
llm_math_chain = LLMMathChain(llm=bedrock_llm, verbose=True)
calculator_tool = Tool(
    name="Calculator",
    func=llm_math_chain.run,
    description="Useful for math calculations.",
)

# Step 3: Define Tools for the Agent
tools = [calculator_tool]

# Step 4: Create the Agent
agent = initialize_agent(
    tools,
    bedrock_llm,
    agent="zero-shot-react-description",
    verbose=True,
)

# Step 5: Run the Agent
response = agent.run("What is the square root of 16 plus 10?")
print(response)



/tmp/ipykernel_24783/874445191.py:21: LangChainDeprecationWarning: This class is deprecated and will be removed in langchain 1.0. See API reference for replacement: https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html
  llm_math_chain = LLMMathChain(llm=bedrock_llm, verbose=True)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:175: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(
/tmp/ipykernel_24783/874445191.py:32: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more



> Entering new AgentExecutor chain...
Thought: To find the square root of 16 plus 10, I first need to calculate the square root of 16, and then add 10 to the result.
Action: Calculator
Action Input: sqrt(16)

> Entering new LLMMathChain chain...
sqrt(16)Question: sqrt(16)
```text
sqrt(16)
```
...numexpr.evaluate("sqrt(16)")...


ValueError: unknown format from LLM: Question: sqrt(16)
```text
sqrt(16)
```
...numexpr.evaluate("sqrt(16)")...

### getting agent into prepared state

In [ ]:
#import helper functions
from helper import *

# Wait for the agent status to become 'NOT_PREPARED'
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='NOT_PREPARED'
)

In [ ]:
# Prepare the agent -> status = preparing
bedrock_agent.prepare_agent(
    agentId=agentId
)

In [ ]:
# Wait for the agent status to become 'PREPARED'
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

In [ ]:
# Create an alias for the agent - invoke agent in order to interact with agent
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='RetailAgent',
)

# Retrieve the alias ID
agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

# Wait for the agent alias status to become 'PREPARED'
wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

In [ ]:
# Initialize the Bedrock Agent Runtime client
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-east-1')

# Test the agent with a message
import uuid
message = "Look into the stocks of bread"

# Create a unique session ID
sessionId = str(uuid.uuid4())

# Invoke the agent and retrieve the response
invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True,
)

# Print the event stream
event_stream = invoke_agent_response["completion"]
for event in event_stream:
    print(event)

# Simplified invoke function
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)

# AI agents in LangGraph course

## build agent from scratch

In [16]:
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()

chat_completion = bedrock_llm.chat.completions.create(
    model=modelId,
    messages=[{"role": "user", "content": "Hello world"}]
)

chat_completion.choices[0].message.content

AttributeError: 'ChatBedrock' object has no attribute 'chat'

In [ ]:
class Agent:
    def __init__(self, system=""):
        self.system = system #save initial system message as parameter
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message}) 
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self): #call LLM
        completion = bedrock_llm.chat.completions.create(
                        model=modelId,
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content

    
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

def calculate(what):
    return eval(what) #evaluates string

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [ ]:
abot = Agent(prompt)
result = abot("How much does a toy poodle weigh?")
print(result)

result = average_dog_weight("Toy Poodle")
result
next_prompt = "Observation: {}".format(result)
abot(next_prompt)
abot.messages

abot = Agent(prompt)
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)
abot(next_prompt)

next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)
abot(next_prompt)

next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)
abot(next_prompt)

# Add loop
action_re = re.compile('^Action: (\w+): (.*)$')   # Python regular expression to select action

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)


# Tests

In [6]:
!pip install langchain openai

from langchain import LangChain
from langchain.llms import Claude

llm = Claude(api_key="YOUR_API_KEY")
lc = LangChain(llm)

from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate(
    system_prompt="You are a friendly chatbot.",
    user_prompt="What's your name?"
)

def handle_user_input(user_input):
    response = lc.generate_response(prompt_template, user_input)
    return response

user_input = input("What's your name?")
response = handle_user_input(user_input)
print(response)

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    response = handle_user_input(user_input)
    print(f"Bot: {response}")

    
from langchain.memory import Memory

memory = Memory()
lc.set_memory(memory)

def handle_user_input(user_input):
    response = lc.generate_response(prompt_template, user_input, memory=memory)
    return response



ImportError: cannot import name 'LangChain' from 'langchain' (/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain/__init__.py)